# <font color=Red># Prologue : CNNs implementation without Torch </font>

해당 프로젝트에서는 PyTorch 없이 CNNs를 구현하고, </br>
간단한 이미지(MNIST 데이터세트)를 분류하는 Classification Task를 수행합니다. </br>
해당 프로젝트의 Experiment Setting은 CNNs Implementation with Torch를 참고합니다. </br> 
</br>
### Preliminary 
모든 Neural Networks는 세 개의 요소로 구성되어 있습니다. </br>
- ```Architecture``` : Neuron과 Layer를 Initialization하고, Forward Propagation을 수행합니다.</br>
- ```Cost function``` : Model과 Label간의 차이를 계산합니다. </br>
- ```Optimization``` : Backward Propagation을 수행하고, weight와 bias로 구성된 Parameter를 Update합니다. </br>

이 세 가지의 구성요소를 코드로 구현하는 것은 매우 복잡하고 지루한 과정이기 때문에, </br>
자주 사용되는 method는 ```Tensorflow```나 ```PyTorch```와 같은 Library에서 제공되는 것을 사용하는 것이 일반적입니다. </br>
</br>
그러나 만약 Library를 사용하지 않고 CNNs를 구현한다면, 어떤 부분이 기존 Library에서 제공되었는지를 파악해야 합니다, </br>

In [1]:
import numpy as np
import cupy as cp
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# <font color=Red># Modulization </font>

먼저, 신경망에 사용되는 여러 가지 함수들을 ```Module```화 하는 것이 중요합니다. </br>
여기서 Module이란 여러 학문 분야에서 사용하는 용어인데, Computer Science에서 사용되는 경우 </br>
특정 프로그램을 구성하는 기본 단위를 의미합니다. </br>
</br>
신경망을 구현하는 데 필요한 각종 Module들은 앞서 설명되었듯이, 여러 Library에서 제공되지만 </br>
해당 프로젝트에서는 Torch 없이 Module들을 구현하는 것을 목표로 하므로, 어떤 Module이 사용되는지 먼저 살펴봅니다. </br>
신경망에서 자주 사용되는 Module들에는 다음과 같은 것들이 있습니다. </br>

### Convolution
가장 먼저, 이미지의 각종 특징(Feature)들을 추출하는 ```Convolution``` 연산을 수행합니다.</br>
Convolution은, ```Filter``` 혹은 ```Kernel```이라 불리는 Matrix를 원본 이미지에 곱함으로써 수행됩니다.</br>
이때, Filter Size는 ```VGGNet```의 설정을 참고하여, 모든 레이어에서 3x3으로 고정합니다.

**Ref:Very Deep Convolutional Networks for Large-Scale Image Recognition**

In [2]:
import cupy as cp

def im2col(input, filter_height, filter_width, stride, padding):
    batch_size, in_channel, in_height, in_width = input.shape
    out_height = (in_height - filter_height + 2 * padding) // stride + 1
    out_width = (in_width - filter_width + 2 * padding) // stride + 1
    
    padded_input = cp.pad(input, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
    
    i0 = cp.repeat(cp.arange(filter_height), filter_width)
    i0 = cp.tile(i0, in_channel)
    i1 = stride * cp.repeat(cp.arange(out_height), out_width)
    j0 = cp.tile(cp.arange(filter_width), filter_height * in_channel)
    j1 = stride * cp.tile(cp.arange(out_width), out_height)
    
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    
    k = cp.repeat(cp.arange(in_channel), filter_height * filter_width).reshape(-1, 1)
    
    cols = padded_input[:, k, i, j]
    cols = cols.transpose(1, 2, 0).reshape(filter_height * filter_width * in_channel, -1)
    return cols

def col2im(cols, input_shape, filter_height, filter_width, stride, padding):
    batch_size, in_channel, in_height, in_width = input_shape
    padded_height = in_height + 2 * padding
    padded_width = in_width + 2 * padding
    padded_input = cp.zeros((batch_size, in_channel, padded_height, padded_width))
    
    out_height = (in_height - filter_height + 2 * padding) // stride + 1
    out_width = (in_width - filter_width + 2 * padding) // stride + 1
    
    i0 = cp.repeat(cp.arange(filter_height), filter_width)
    i0 = cp.tile(i0, in_channel)
    i1 = stride * cp.repeat(cp.arange(out_height), out_width)
    j0 = cp.tile(cp.arange(filter_width), filter_height * in_channel)
    j1 = stride * cp.tile(cp.arange(out_width), out_height)
    
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    
    k = cp.repeat(cp.arange(in_channel), filter_height * filter_width).reshape(-1, 1)
    
    cols_reshaped = cols.reshape(in_channel * filter_height * filter_width, -1, batch_size)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    
    cp.add.at(padded_input, (slice(None), k, i, j), cols_reshaped)
    
    if padding == 0:
        return padded_input
    return padded_input[:, :, padding:-padding, padding:-padding]

def conv2d(input, filters, bias, stride=1, padding=1):
    batch_size, in_channel, in_height, in_width = input.shape
    out_channel, _, filter_height, filter_width = filters.shape
    
    out_height = (in_height - filter_height + 2 * padding) // stride + 1
    out_width = (in_width - filter_width + 2 * padding) // stride + 1
    
    cols = im2col(input, filter_height, filter_width, stride, padding)
    reshaped_filters = filters.reshape(out_channel, -1)
    
    out = reshaped_filters @ cols + bias.reshape(-1, 1)
    out = out.reshape(out_channel, out_height, out_width, batch_size)
    out = out.transpose(3, 0, 1, 2)
    return out

def conv2d_backward(dout, input, filters, stride=1, padding=1):
    batch_size, in_channel, in_height, in_width = input.shape
    out_channel, _, filter_height, filter_width = filters.shape
    
    cols = im2col(input, filter_height, filter_width, stride, padding)
    dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(out_channel, -1)
    
    dfilters = dout_reshaped @ cols.T
    dfilters = dfilters.reshape(filters.shape)
    
    dbias = cp.sum(dout, axis=(0, 2, 3))
    
    dcols = filters.reshape(out_channel, -1).T @ dout_reshaped
    dinput = col2im(dcols, input.shape, filter_height, filter_width, stride, padding)
    
    return dinput, dfilters, dbias


### Pooling
해당 이미지의 특정 영역의 값을 압축하는 ```Pooling```연산을 수행합니다.</br>
Classification Task에서는 다른 Pooling 방법에 비해 특정 영역을 해당 영역의 최댓값으로 압축하는 ```Max Polling```이 일반적으로 더 좋은 성능을 보여주므로,</br>
해당 코드에서도 Max Pooling을 기본적으로 구현합니다.</br>
Max Pooling 역시 Convolution과 동일하게 Pooling Kernel을 설정하지만 </br>
Convoltuion과는 다르게 Parameter를 저장할 필요가 없습니다.

In [3]:
def max_pool2d(input, pool_size=2, stride=2):
    batch_size, channel, height, width = input.shape  # 입력 텐서의 형태 (배치 크기, 채널 수, 높이, 너비) 구함
    out_height = (height - pool_size) // stride + 1  # 출력 텐서의 높이 계산
    out_width = (width - pool_size) // stride + 1  # 출력 텐서의 너비 계산
    output = cp.zeros((batch_size, channel, out_height, out_width))  # 출력 텐서 초기화
    max_indices = cp.zeros_like(input)  # 최대값 인덱스 초기화

    # 출력 텐서의 각 위치에 대해 최대 풀링 수행
    for i in range(out_height):
        for j in range(out_width):
            h_start = i * stride  # 입력 텐서에서 현재 위치의 시작 높이
            h_end = h_start + pool_size  # 입력 텐서에서 현재 위치의 끝 높이
            w_start = j * stride  # 입력 텐서에서 현재 위치의 시작 너비
            w_end = w_start + pool_size  # 입력 텐서에서 현재 위치의 끝 너비
            region = input[:, :, h_start:h_end, w_start:w_end]  # 입력 텐서에서 현재 위치의 영역 추출
            output[:, :, i, j] = cp.max(region, axis=(2, 3))  # 영역 내의 최대값 계산하여 출력 텐서에 저장
            max_indices[:, :, h_start:h_end, w_start:w_end] += (region == output[:, :, i:i+1, j:j+1])  # 최대값 위치 저장

    return output, max_indices  # 최대 풀링 결과와 최대값 인덱스 반환

def max_pool2d_backward(dout, max_indices, pool_size=2, stride=2):
    batch_size, channel, height, width = max_indices.shape  # 최대값 인덱스의 형태 (배치 크기, 채널 수, 높이, 너비) 구함
    dinput = cp.zeros_like(max_indices)  # 입력에 대한 변화도 초기화

    out_height = dout.shape[2]  # dout의 높이 구함
    out_width = dout.shape[3]  # dout의 너비 구함

    # 역전파 수행
    for i in range(out_height):
        for j in range(out_width):
            h_start = i * stride  # 입력 텐서에서 현재 위치의 시작 높이
            h_end = h_start + pool_size  # 입력 텐서에서 현재 위치의 끝 높이
            w_start = j * stride  # 입력 텐서에서 현재 위치의 시작 너비
            w_end = w_start + pool_size  # 입력 텐서에서 현재 위치의 끝 너비
            dinput[:, :, h_start:h_end, w_start:w_end] += (max_indices[:, :, h_start:h_end, w_start:w_end] * dout[:, :, i:i+1, j:j+1])  # 최대값 위치에 dout 적용

    return dinput  # 입력에 대한 변화도 반환

### Activation Function
- Sigmoid, ReLU, Softmax
기본적으로 Activation Function으로는 ```ReLU```를 사용하고, </br>
Classification Task를 수행하기 때문에</br>
Classifier의 마지막 Layer에서는 ```Softmax```를 사용합니다. 


In [4]:
def relu(x):
    return cp.maximum(0, x)

def relu_backward(dout, x):
    return dout * (x > 0)

**Convolution과 Pooling으로 구성된 Feature Extractor의 설정은 다음과 같습니다.**

|Layer|in_channel|out_chnnel|kernel_size|stride|padding|
|---|---|---|---|---|---|
|Conv1|1|32|3|1|1|
|Conv2|32|32|3|1|1|
|Pool1|2|2|0|
|Conv3|32|128|1|1|1|
|Conv4|128|128|3|1|1|
|Pool2|2|2|0|
|Conv5|128|256|3|1|1|
|Conv6|256|256|3|1|1|
|Pool3|2|2|

### Linear Combination(FC layer)
Feature extractor에서 생성한 Feature의 정보를 모아서 분류를 수행하는 Classifier를 정의합니다. </br>
Classifier에서는 Feature의 정보를 모으기 위해 Convolutional Layer를 Flatten(Vectorization)하기 때문에 </br>
각 Neuron들을 ```Linear Combination```으로 연결하는 ```Fully Connected(FC) Layer```를 구현해야 합니다. </br>

**Classifier의 설정은 다음과 같습니다.**

|Layer|in_features|out_features|
|---|---|---|
|FC1|3x3x256|4096|
|Dropout(0.5)|
|FC2|4096|10|

In [5]:
def flatten(input):
    return input.reshape(input.shape[0], -1)  # 입력 텐서를 평탄화하여 2차원 텐서로 변환 (배치 크기, 나머지 차원)

def fully_connected(input, weights, bias):
    return cp.dot(input, weights) + bias  # 입력 텐서와 가중치를 행렬 곱셈하고 바이어스를 더하여 선형 변환 수행

def fully_connected_backward(dout, input, weights):
    dinput = cp.dot(dout, weights.T)  # dout과 가중치의 전치를 곱하여 입력에 대한 변화도 계산
    dweights = cp.dot(input.T, dout)  # 입력의 전치와 dout을 곱하여 가중치에 대한 변화도 계산
    dbias = cp.sum(dout, axis=0)  # dout의 모든 배치에 대해 합을 구하여 바이어스에 대한 변화도 계산
    return dinput, dweights, dbias  # 입력, 가중치, 바이어스에 대한 변화도 반환

### Dropout
더 이상의 자세한 설명은 생략한다.

# Batch Normalization

In [6]:
class BatchNormalization:
    def __init__(self, num_features, epsilon=1e-5, momentum=0.9):
        self.num_features = num_features
        self.epsilon = epsilon
        self.momentum = momentum
        self.gamma = cp.ones((1, num_features, 1, 1))
        self.beta = cp.zeros((1, num_features, 1, 1))
        self.running_mean = cp.zeros((1, num_features, 1, 1))
        self.running_var = cp.ones((1, num_features, 1, 1))
        self.dgamma = cp.zeros_like(self.gamma)
        self.dbeta = cp.zeros_like(self.beta)
    
    def forward(self, x, train=True):
        if train:
            mean = cp.mean(x, axis=(0, 2, 3), keepdims=True)
            var = cp.var(x, axis=(0, 2, 3), keepdims=True)
            self.x_centered = x - mean
            self.stddev_inv = 1. / cp.sqrt(var + self.epsilon)
            x_norm = self.x_centered * self.stddev_inv
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var
            out = self.gamma * x_norm + self.beta
            return out
        else:
            x_norm = (x - self.running_mean) / cp.sqrt(self.running_var + self.epsilon)
            return self.gamma * x_norm + self.beta
    
    def backward(self, dout):
        N, C, H, W = dout.shape
        
        x_norm = self.x_centered * self.stddev_inv
        self.dbeta = cp.sum(dout, axis=(0, 2, 3), keepdims=True)
        self.dgamma = cp.sum(dout * x_norm, axis=(0, 2, 3), keepdims=True)
        
        dx_norm = dout * self.gamma
        dvar = cp.sum(dx_norm * self.x_centered * -0.5 * self.stddev_inv**3, axis=(0, 2, 3), keepdims=True)
        dmean = cp.sum(dx_norm * -self.stddev_inv, axis=(0, 2, 3), keepdims=True) + dvar * cp.mean(-2. * self.x_centered, axis=(0, 2, 3), keepdims=True)
        
        dx = (dx_norm * self.stddev_inv) + (dvar * 2 * self.x_centered / N) + (dmean / N)
        
        return dx
    def update_params(self, optimizer, layer_index):
        optimizer.update_params(layer_index, self.gamma, self.dgamma)
        optimizer.update_params(layer_index + 1, self.beta, self.dbeta)


# <font color=Red># Class </font>

Python에서는 Class를 이용하여 Modulization을 수행할 수 있습니다. </br>
이때, 중요한 점은 해당 Class의 method는, 각 module의 ```Forward Propagation``` 부분과</br> 
```Backpropagation```이 모두 구현되어 있어야 한다는 점입니다.</br>

따라서, 모든 클래스에는 Forward Propagation method에서 해당 함수의 연산을 수행하는 부분을 정의하고, </br>
Backpropagation method에서 해당 함수의 ```gradient```를 계산하는 연산을 수행하는 부분을 정의합니다. 

### Dropout Module

#### Forward Propagation
define p : dropout probability </br>

$$\text{mask} = \frac{\mathbf{r}}{1-p}$$
r = uniformly distributed matrix for range [0, 1]

- **During Training**
$$\mathbf{y} = \mathbf{x} \odot \text{mask}$$
여기서, $\mathbf{y}$는 Dropout이 적용된 출력, $\mathbf{x}$는 입력, $\odot$는 element-wise multiplication을 나타냄.

- **During Test**
$$\mathbf{y} = \mathbf{x}$$
Test 시에는 Dropout이 적용되지 않음.

#### Backward Propagation
$$\mathbf{dout} = \mathbf{dout} \odot \text{mask}$$

In [7]:
class Dropout:
    def __init__(self, p=0.5):
        self.p = p  # 드롭아웃 확률 설정
        self.mask = None  # 마스크 초기화

    def forward(self, input, train=True):
        if train:  # 학습 중일 때
            self.mask = (cp.random.rand(*input.shape) > self.p) / (1 - self.p)  # 드롭아웃 마스크 생성
            return input * self.mask  # 입력에 마스크 적용하여 드롭아웃 수행
        else:  # 평가 중일 때
            return input  # 입력 그대로 반환

    def backward(self, dout):
        return dout * self.mask  # 역전파 시 마스크 적용하여 그래디언트 계산


### Convolution Module

In [8]:
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        self.stride = stride  # 스트라이드 값 설정
        self.padding = padding  # 패딩 값 설정
        # He 초기화를 사용하여 필터 가중치 초기화
        self.weights = cp.random.randn(out_channels, in_channels, kernel_size, kernel_size) * cp.sqrt(2 / (in_channels * kernel_size * kernel_size))
        self.bias = cp.zeros(out_channels)  # 바이어스 초기화
        self.dweights = cp.zeros_like(self.weights)  # 필터 가중치 변화도 초기화
        self.dbias = cp.zeros_like(self.bias)  # 바이어스 변화도 초기화

    def forward(self, input):
        self.input = input  # 입력값 저장 (역전파를 위해)
        # 컨벌루션 연산 수행 후 ReLU 활성화 함수 적용하여 출력 계산
        self.output = relu(conv2d(input, self.weights, self.bias, self.stride, self.padding))
        return self.output  # 최종 출력 반환

    def backward(self, dout):
        # ReLU의 역전파 수행
        dout = relu_backward(dout, self.output)
        # 컨벌루션 연산의 역전파 수행하여 입력, 필터 가중치, 바이어스에 대한 변화도 계산
        dinput, self.dweights, self.dbias = conv2d_backward(dout, self.input, self.weights, self.stride, self.padding)
        return dinput  # 입력에 대한 변화도 반환
    
    def update_params(self, optimizer, layer_index):
        optimizer.update_params(layer_index, self.weights, self.dweights)
        optimizer.update_params(layer_index + 1, self.bias, self.dbias)

### Maxpool Module

In [9]:
class MaxPool2D:
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size  # 풀링 크기 설정
        self.stride = stride  # 스트라이드 값 설정

    def forward(self, input):
        self.input = input  # 입력값 저장 (역전파를 위해)
        # 최대 풀링 연산 수행, 출력값과 최대값 인덱스 저장
        self.output, self.max_indices = max_pool2d(input, self.pool_size, self.stride)
        return self.output  # 최종 출력 반환

    def backward(self, dout):
        # 최대 풀링의 역전파 수행하여 입력에 대한 변화도 계산
        return max_pool2d_backward(dout, self.max_indices, self.pool_size, self.stride)

### ReLU Module

In [10]:
class ReLU:
    @staticmethod
    def forward(x):
        return relu(x)

    @staticmethod
    def backward(dout, x):
        return relu_backward(dout, x)

### Vectorization Module

In [11]:
class Flatten:
    def forward(self, input):
        self.input_shape = input.shape
        output = flatten(input)
        return output

    def backward(self, dout):
        return dout.reshape(self.input_shape)


### Linear Module

In [12]:
class FullyConnected:
    def __init__(self, in_features, out_features):
        # He initialization을 사용하여 가중치 초기화
        self.weights = cp.random.randn(in_features, out_features) * cp.sqrt(2 / in_features)
        self.bias = cp.zeros(out_features)  # 바이어스 초기화
        self.dweights = cp.zeros_like(self.weights)  # 가중치 변화도 초기화
        self.dbias = cp.zeros_like(self.bias)  # 바이어스 변화도 초기화

    def forward(self, input):
        self.input = input  # 입력값 저장 (역전파를 위해)
        self.output = fully_connected(input, self.weights, self.bias)  # 선형 변환 수행
        return relu(self.output)  # ReLU 활성화 함수 적용 후 반환

    def backward(self, dout):
        dout = relu_backward(dout, self.output)  # ReLU의 역전파 수행
        # 선형 변환의 역전파 수행하여 입력, 가중치, 바이어스에 대한 변화도 계산
        dinput, self.dweights, self.dbias = fully_connected_backward(dout, self.input, self.weights)
        return dinput  # 입력에 대한 변화도 반환
    def update_params(self, optimizer, layer_index):
        optimizer.update_params(layer_index, self.weights, self.dweights)
        optimizer.update_params(layer_index + 1, self.bias, self.dbias)

### MC Dropout for ESC

In [35]:
def mc_dropout_predict(test_batches, model, num_samples=10):
    predictions = []

    for sample_idx in range(num_samples):
        batch_predictions = []
        for input, _ in test_batches:
            output = model.forward(input, train=True)
            output = softmax(output)
            batch_predictions.append(output)
        
        batch_predictions = cp.concatenate(batch_predictions, axis=0)
        predictions.append(batch_predictions)

    mean_predictions = cp.mean(cp.stack(predictions), axis=0)
    final_predictions = cp.argmax(mean_predictions, axis=1)
    
    return final_predictions



# <font color=Red># Architecture </font>

In [14]:
class SimpleCNN:
    def __init__(self):
        self.layers = [
            Conv2D(1, 32, 3, stride=1, padding=1),  # 입력 채널 1, 출력 채널 32, 커널 크기 3x3
            BatchNormalization(32),
            MaxPool2D(2, 2),  # 최대 풀링, 풀링 크기 2x2
            Conv2D(32, 64, 3, stride=1, padding=1),  # 입력 채널 32, 출력 채널 64, 커널 크기 3x3
            BatchNormalization(64),
            MaxPool2D(2, 2),  # 최대 풀링, 풀링 크기 2x2
            Flatten(),  # 평탄화
            FullyConnected(64 * 7 * 7, 128),  # 완전 연결 계층
            Dropout(0.5),  # 드롭아웃
            FullyConnected(128, 10)  # 출력 클래스 수 10개
        ]

    def forward(self, x, train=True):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train)
            else:
                x = layer.forward(x)
        return x
    def backward(self, dout):
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def get_params(self):
        params = []
        for layer in self.layers:
            if isinstance(layer, (Conv2D, FullyConnected)):
                params.append({'value': layer.weights, 'grad': layer.dweights})
                params.append({'value': layer.bias, 'grad': layer.dbias})
            if isinstance(layer, BatchNormalization):
                params.append({'value': layer.gamma, 'grad': layer.dgamma})
                params.append({'value': layer.beta, 'grad': layer.dbeta})
        return params


# <font color=Red># Cost Function </font>

In [15]:
def cross_entropy_loss(output, target, model, l2_lambda=0.005):
    batch_size = target.shape[0]
    output = cp.clip(output, 1e-12, 1. - 1e-12)
    loss = -cp.sum(target * cp.log(output)) / batch_size
    
    # L2 정규화 항 추가
    l2_loss = 0
    for layer in model.layers:
        if isinstance(layer, (Conv2D, FullyConnected)):
            l2_loss += cp.sum(layer.weights ** 2)
    loss += l2_lambda * l2_loss / 2
    return loss

def cross_entropy_derivative(output, target):
    return output - target

def softmax(x):
    exp_x = cp.exp(x - cp.max(x, axis=1, keepdims=True))
    return exp_x / cp.sum(exp_x, axis=1, keepdims=True)


# <font color=Red># Data Loader </font>

In [16]:
from torch.utils.data import DataLoader, Subset
def load_data(batch_size=32):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # 전체 MNIST 데이터셋을 로드합니다.
    train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('../data', train=False, download=True, transform=transform)
    
    # 훈련 데이터셋에서 10,000개 샘플만 선택합니다.
    train_indices = list(range(60000))
    train_subset = Subset(train_dataset, train_indices)
    
    # 테스트 데이터셋에서 2,000개 샘플만 선택합니다.
    test_indices = list(range(10000))
    test_subset = Subset(test_dataset, test_indices)
    
    # DataLoader를 생성합니다.
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
    
    # DataLoader에서 배치를 가져와 CuPy 배열로 변환합니다.
    train_batches = [(cp.array(data.numpy()), cp.array(labels.numpy())) for data, labels in train_loader]
    test_batches = [(cp.array(data.numpy()), cp.array(labels.numpy())) for data, labels in test_loader]
    
    return train_batches, test_batches

# <font color=Red># Optimization </font>

### ADAM(Adaptive Moment Estimation)

**0. Initialize 1st momentum and 2nd momentum**
$$m_0 = 0 $$ 
$$v_0 = 0 $$ </br>

**1. Update Time step** </br>
$$t = t + 1$$


**2. Update 1st Momentum and 2nd Momentum(using the gradient g)**

$$m_t = \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot g_t$$
$$v_t = \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot g_t^2$$

**3. Update estimation**
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$$
$$\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$

**4. Update parameters(w, b)**

$$theta_t = \theta_{t-1} - \alpha \cdot \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

- Momentum coefficient(Friction)$(\beta_1,\,\,\beta_2)$ , Generally choosen 0.9 and 0.999, respectively.


In [17]:
class Adam:
    def __init__(self, params, lr=0.1, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.params = params  # w, b (parameters to optimize)
        self.lr = lr  # Learning rate
        self.beta1 = beta1  # coefficient of 1st momentum
        self.beta2 = beta2  # coefficient of 2nd momentum
        self.epsilon = epsilon  # avoid to devided by 0
        
        self.m = [cp.zeros_like(p['value']) for p in params]  # initialize 1st momentum
        self.v = [cp.zeros_like(p['value']) for p in params]  # initialize 2nd momentum
        self.t = 0  # initialize time step

    def update_params(self, index, param, grad):
        self.t += 1  # 타임스텝 증가
        # Update 1st momentum
        self.m[index] = self.beta1 * self.m[index] + (1 - self.beta1) * grad
        # Update 2nd momentum
        self.v[index] = self.beta2 * self.v[index] + (1 - self.beta2) * (grad ** 2)
        # update 1st momentum estimation
        m_hat = self.m[index] / (1 - self.beta1 ** self.t)
        # update 2nd momentum estimation
        v_hat = self.v[index] / (1 - self.beta2 ** self.t)
        # update parameters
        param -= self.lr * m_hat / (cp.sqrt(v_hat) + self.epsilon)

In [18]:
class ExponentialLearningRateScheduler:
    def __init__(self, optimizer, decay_rate=0.85):
        self.optimizer = optimizer
        self.decay_rate = decay_rate
        self.initial_lr = optimizer.lr

    def step(self, epoch):
        new_lr = self.initial_lr * (self.decay_rate ** epoch)
        self.optimizer.lr = new_lr
        print(f"Epoch {epoch + 1}, Updated Learning Rate: {self.optimizer.lr}")


In [19]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def __call__(self, current_loss):
        if self.best_loss is None:
            self.best_loss = current_loss
            return False
        
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False


In [20]:
# def train(train_batches, model, optimizer, scheduler, early_stopping, epochs=10):

def train(train_batches, model, optimizer, scheduler, epochs=20):
    for epoch in range(epochs):
        total_loss = 0
        for i, (input, target) in enumerate(train_batches):
            # Forward pass
            output = model.forward(input)
            # Compute loss
            output = softmax(output)
            loss = cross_entropy_loss(output, target, model, l2_lambda=0.0005)
            total_loss += loss
            # Backward pass
            dout = cross_entropy_derivative(output, target)
            model.backward(dout)
            # Update weights and biases for each layer using Adam optimizer
            layer_index = 0
            for layer in model.layers:
                if isinstance(layer, (Conv2D, FullyConnected, BatchNormalization)):
                    layer.update_params(optimizer, layer_index)
                    layer_index += 2  # weights and bias are treated separately
            # Check early stopping condition after each batch
        
#             if early_stopping(loss):
#                 print(f"Early stopping triggered within epoch {epoch + 1}, batch {i + 1}")
#                 break
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss}, Batch Size: {input.shape[0]}, Learning Rate: {optimizer.lr}')
        # Update learning rate
        scheduler.step(epoch)
        print(f'Epoch {epoch + 1}, Average Loss: {total_loss / len(train_batches)}, Learning Rate: {optimizer.lr}')

def test(test_batches, model):
    correct = 0
    total = 0
    for input, target in test_batches:
        output = model.forward(input)
        output = softmax(output)
        predictions = cp.argmax(output, axis=1)
        labels = cp.argmax(target, axis=1)
        correct += cp.sum(predictions == labels)
        total += labels.size
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [32]:
def test_mc(test_batches, model, num_samples=10):
    correct = 0
    total = 0

    for input, target in test_batches:
        output = mc_dropout_predict([(input, target)], model, num_samples=num_samples)
        labels = cp.argmax(target, axis=1)
        correct += cp.sum(output == labels)
        total += labels.size
    
    accuracy = correct / total
    print(f'Test Accuracy with MC Dropout: {accuracy * 100:.2f}%')


In [22]:
def to_one_hot(labels, num_classes=10):
    return cp.eye(num_classes)[labels]

In [23]:
# Initialize model
model = SimpleCNN()

# Load data
train_batches, test_batches = load_data()

# Convert labels to one-hot encoding
train_batches = [(data, to_one_hot(labels)) for data, labels in train_batches]
test_batches = [(data, to_one_hot(labels)) for data, labels in test_batches]

In [24]:
params = model.get_params()
optimizer = Adam(params, lr=0.0003)
scheduler = ExponentialLearningRateScheduler(optimizer, decay_rate=0.85)
early_stopping = EarlyStopping(patience=3, min_delta=1e-4)
# # Train the model
# train(train_batches, model, optimizer, scheduler, early_stopping)

train(train_batches, model, optimizer, scheduler)

Epoch 1, Batch 1, Loss: 3.4628065750427903, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 2, Loss: 4.323901005669167, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 3, Loss: 3.6663169092463104, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 4, Loss: 2.940170927448304, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 5, Loss: 2.769407604313205, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 6, Loss: 2.758892532918841, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 7, Loss: 2.882656745102573, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 8, Loss: 2.29088381969205, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 9, Loss: 2.2242606965334235, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 10, Loss: 2.7911693464005345, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 11, Loss: 1.8991437823788406, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 12, Loss: 2.3105165804421333, Batch Size: 32, Learning Rate: 0.0003
Epoch 1, Batch 13, L

In [29]:
# test(test_batches, model)


In [36]:
test_mc(test_batches, model)

Test Accuracy with MC Dropout: 97.97%


In [34]:
# lambda : 0.005 and alpha : 0.0003 : 93.78
# Lambda : 0.0005 and alpha : 0.0003 : 94.59 epoch 20 : 94.89 